In [16]:
import nltk
import datetime as datetime 
import pickle 
import pandas as pd 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [19]:
mw_dataset_no_dup_no_troll = pd.read_csv('SG_twitter_complete_dataset.csv', encoding='latin1')

In [25]:
len(mw_dataset_no_dup_no_troll)

366625

In [20]:
mw_dataset_no_dup_no_troll.head()

,ID,keywords,tweet_ID,tweet_timestamp,Tweet,retweet_count,favourite_count,HashTags,anger,fear,sadness,joy,valence,user_id,user_joining_date,user_status_count,user_followers,user_friends,location
0,1,wuhan,1224742687137419264,2020-02-05 01:13:05,Coronavirus outbreak: Singapore to provide S$1...,0,0,NaN,0.342,0.508,0.394,0.290,0.517,3099154459,2015-03-20 15:42:35,9415,150,103,Singapore
1,2,wuhan,1224741485859557378,2020-02-05 01:08:19,Coronavirus: Royal Caribbean warns of more cru...,0,0,NaN,0.452,0.636,0.467,0.238,0.400,1200474236,2013-02-20 18:48:39,30905,22711,72,Singapore
2,3,wuhan,1224739548690694144,2020-02-05 01:00:37,The Wuhan Coronavirus Poses Three Tests for Gl...,0,0,WuhanCoronavirus,0.334,0.460,0.349,0.331,0.480,1117748674090504195,2019-04-15 19:17:05,1776,90,638,Singapore
3,4,wuhan,1224738837198278656,2020-02-05 00:57:47,@asadowaisi Pakistan is not rescuing their peo...,0,0,NaN,0.405,0.422,0.432,0.230,0.428,110612755,2010-02-02 12:39:06,4891,280,346,Singapore
4,5,wuhan,1224738184614907904,2020-02-05 00:55:12,The world will pay a growth price for the Wuha...,0,1,NaN,0.335,0.412,0.337,0.352,0.522,1117748674090504195,2019-04-15 19:17:05,1776,90,638,Singapore


In [21]:
mw_dataset_no_dup_no_troll.columns

Index(['ID', 'keywords', 'tweet_ID', 'tweet_timestamp', 'Tweet',
       'retweet_count', 'favourite_count', 'HashTags', 'anger', 'fear',
       'sadness', 'joy', 'valence', 'user_id', 'user_joining_date',
       'user_status_count', 'user_followers', 'user_friends', 'location'],
      dtype='object')

In [22]:
mw_dataset_no_dup_no_troll['Tweet'] = mw_dataset_no_dup_no_troll['Tweet'].astype(str)

In [31]:
import inflect
import re
import string
import unicodedata

def clean_text_1(headline):
    headline = headline.lower()
    headline = re.sub('\[.,*?\]','',headline)
    headline = re.sub('[0-9]','',headline)
    headline = re.sub('[%s]' % re.escape(string.punctuation), '', headline)
    headline = re.sub('[‘’“”…–]', '', headline)
    headline = re.sub('\w*\d\n\w*', '', headline)
    return headline

#def replace_contractions(text):
    #"""Replace contractions in string of text"""
    #return contractions.fix(text)

def remove_mentions(sample):
    """Remove mentions from a sample string"""
    return re.sub(r"@\S+", "", str(sample))
    
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", str(sample))

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', str(word))
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words


def normalize(words):
    #words = replace_numbers(words)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample= remove_mentions(sample)
    #sample = replace_contractions(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)

    # Normalize
    return normalize(words)


round_1 = lambda x: preprocess(x)

In [32]:
mw_dataset_no_dup_no_troll['processed_content'] = mw_dataset_no_dup_no_troll['Tweet'].apply(round_1)

In [33]:
mw_dataset_no_dup_no_troll['processed_content']=mw_dataset_no_dup_no_troll['processed_content'].apply(' '.join)

In [34]:
mw_dataset_no_dup_no_troll['processed_content'].head()

0    coronavirus outbreak singapore to provide s 1 ...
1    coronavirus royal caribbean warns of more crui...
2    the wuhan coronavirus poses three tests for gl...
3    pakistan is not rescuing their people from wuh...
4    the world will pay a growth price for the wuha...
Name: processed_content, dtype: object

In [35]:

mw_dataset_no_dup_no_troll.head()

,ID,keywords,tweet_ID,tweet_timestamp,Tweet,retweet_count,favourite_count,HashTags,anger,fear,sadness,joy,valence,user_id,user_joining_date,user_status_count,user_followers,user_friends,location,processed_content,lemmatized_content
0,1,wuhan,1224742687137419264,2020-02-05 01:13:05,Coronavirus outbreak: Singapore to provide S$1...,0,0,NaN,0.342,0.508,0.394,0.290,0.517,3099154459,2015-03-20 15:42:35,9415,150,103,Singapore,coronavirus outbreak singapore to provide s 1 ...,coronavirus outbreak singapor to provid s 1 mi...
1,2,wuhan,1224741485859557378,2020-02-05 01:08:19,Coronavirus: Royal Caribbean warns of more cru...,0,0,NaN,0.452,0.636,0.467,0.238,0.400,1200474236,2013-02-20 18:48:39,30905,22711,72,Singapore,coronavirus royal caribbean warns of more crui...,coronavirus royal caribbean warn of more cruis...
2,3,wuhan,1224739548690694144,2020-02-05 01:00:37,The Wuhan Coronavirus Poses Three Tests for Gl...,0,0,WuhanCoronavirus,0.334,0.460,0.349,0.331,0.480,1117748674090504195,2019-04-15 19:17:05,1776,90,638,Singapore,the wuhan coronavirus poses three tests for gl...,the wuhan coronavirus pose three test for glob...
3,4,wuhan,1224738837198278656,2020-02-05 00:57:47,@asadowaisi Pakistan is not rescuing their peo...,0,0,NaN,0.405,0.422,0.432,0.230,0.428,110612755,2010-02-02 12:39:06,4891,280,346,Singapore,pakistan is not rescuing their people from wuh...,sadowaisi pakistan is not rescu their peopl fr...
4,5,wuhan,1224738184614907904,2020-02-05 00:55:12,The world will pay a growth price for the Wuha...,0,1,NaN,0.335,0.412,0.337,0.352,0.522,1117748674090504195,2019-04-15 19:17:05,1776,90,638,Singapore,the world will pay a growth price for the wuha...,the world will pay a growth price for the wuha...


In [36]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

#wordnet_lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

#fb_clean['processed_content'] = fb_clean['processed_content'].astype('str').apply(lambda x: filter(None,x.split(" ")))
#fb_clean['processed_content'] = fb_clean['processed_content'].apply(lambda x : [wordnet_lemmatizer.lemmatize(y) for y in x])
#fb_clean['processed_content']=fb_clean['processed_content'].apply(lambda x : " ".join(x))

mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['processed_content'].astype('str').apply(lambda x: filter(None,x.split(" ")))
#df['processed_content'] = df['processed_content'].apply(lambda x : [wordnet_lemmatizer.lemmatize(y) for y in x])
mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['lemmatized_content'].apply(lambda x : [stemmer.stem(y) for y in x])
mw_dataset_no_dup_no_troll['lemmatized_content'] = mw_dataset_no_dup_no_troll['lemmatized_content'].apply(lambda x : " ".join(x))

In [37]:
mw_dataset_no_dup_no_troll.head()

,ID,keywords,tweet_ID,tweet_timestamp,Tweet,retweet_count,favourite_count,HashTags,anger,fear,sadness,joy,valence,user_id,user_joining_date,user_status_count,user_followers,user_friends,location,processed_content,lemmatized_content
0,1,wuhan,1224742687137419264,2020-02-05 01:13:05,Coronavirus outbreak: Singapore to provide S$1...,0,0,NaN,0.342,0.508,0.394,0.290,0.517,3099154459,2015-03-20 15:42:35,9415,150,103,Singapore,coronavirus outbreak singapore to provide s 1 ...,coronavirus outbreak singapor to provid s 1 mi...
1,2,wuhan,1224741485859557378,2020-02-05 01:08:19,Coronavirus: Royal Caribbean warns of more cru...,0,0,NaN,0.452,0.636,0.467,0.238,0.400,1200474236,2013-02-20 18:48:39,30905,22711,72,Singapore,coronavirus royal caribbean warns of more crui...,coronavirus royal caribbean warn of more cruis...
2,3,wuhan,1224739548690694144,2020-02-05 01:00:37,The Wuhan Coronavirus Poses Three Tests for Gl...,0,0,WuhanCoronavirus,0.334,0.460,0.349,0.331,0.480,1117748674090504195,2019-04-15 19:17:05,1776,90,638,Singapore,the wuhan coronavirus poses three tests for gl...,the wuhan coronavirus pose three test for glob...
3,4,wuhan,1224738837198278656,2020-02-05 00:57:47,@asadowaisi Pakistan is not rescuing their peo...,0,0,NaN,0.405,0.422,0.432,0.230,0.428,110612755,2010-02-02 12:39:06,4891,280,346,Singapore,pakistan is not rescuing their people from wuh...,pakistan is not rescu their peopl from wuhan i...
4,5,wuhan,1224738184614907904,2020-02-05 00:55:12,The world will pay a growth price for the Wuha...,0,1,NaN,0.335,0.412,0.337,0.352,0.522,1117748674090504195,2019-04-15 19:17:05,1776,90,638,Singapore,the world will pay a growth price for the wuha...,the world will pay a growth price for the wuha...


In [38]:
mw_dataset_no_dup_no_troll.to_csv('SG_twitter_preprocessed_V2_26AUG.csv')

In [15]:
len(mw_dataset_no_dup_no_troll)

1674662